### initialize env & comfyui lib ###

In [4]:
!pip3 install websocket-client
!pip3 install -U requests

In [8]:
sagemaker_default_bucket

'sagemaker-us-west-2-687912291502'

In [7]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### funcs for comfyui

In [4]:
import websocket
import uuid
import json
import urllib.request
import urllib.parse
import base64

#server_address = "ec2-34-222-223-235.us-west-2.compute.amazonaws.com:8188"
server_address = "ec2-35-91-44-198.us-west-2.compute.amazonaws.com:8188"
client_id = str(uuid.uuid4())

def queue_prompt(prompt):
    print(prompt)
    p = {"prompt": prompt, "client_id": client_id}
    data = json.dumps(p).encode('utf-8')
    url = "http://"+server_address+"/prompt"
    req = urllib.request.Request(url, data=data)
    #req =  urllib.request.Request("http://ec2-34-222-223-235.us-west-2.compute.amazonaws.com:8188/prompt", data=data)
    return json.loads(urllib.request.urlopen(req).read())

def get_image_privew(filename):
    url = "http://{}/view?filename={}&type=output".format(server_address,filename)
    with urllib.request.urlopen(url) as response:
        return response.read()

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    print("prompt_id=="+prompt_id)
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        print("output==")
        print(o)
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    #image_data = get_image_privew(image['filename'])
                    print("image data==\n")
                    #image_text = (image_data).decode('utf-8') 
                    print(image_data)
                    images_output.append(image_data)
                output_images[node_id] = images_output
            # video branch
            if 'gifs' in node_output:
                videos_output = []
                for video in node_output['gifs']:
                    video_data = get_image(video['filename'], video['subfolder'], video['type'])
                    videos_output.append(video_data)
                output_images[node_id] = videos_output

    return output_images

### 生图 test

In [2]:
prompt_text = """
{
    "3": {
        "class_type": "KSampler",
        "inputs": {
            "cfg": 8,
            "denoise": 1,
            "latent_image": [
                "5",
                0
            ],
            "model": [
                "4",
                0
            ],
            "negative": [
                "7",
                0
            ],
            "positive": [
                "6",
                0
            ],
            "sampler_name": "euler",
            "scheduler": "normal",
            "seed": 8566257,
            "steps": 20
        }
    },
    "4": {
        "class_type": "CheckpointLoaderSimple",
        "inputs": {
            "ckpt_name": "cardosAnime_v20.safetensors"
        }
    },
    "5": {
        "class_type": "EmptyLatentImage",
        "inputs": {
            "batch_size": 1,
            "height": 512,
            "width": 512
        }
    },
    "6": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "clip": [
                "4",
                1
            ],
            "text": "masterpiece best quality girl,naked"
        }
    },
    "7": {
        "class_type": "CLIPTextEncode",
        "inputs": {
            "clip": [
                "4",
                1
            ],
            "text": "bad hands"
        }
    },
    "8": {
        "class_type": "VAEDecode",
        "inputs": {
            "samples": [
                "3",
                0
            ],
            "vae": [
                "4",
                2
            ]
        }
    },
    "9": {
        "class_type": "SaveImage",
        "inputs": {
            "filename_prefix": "ComfyUI",
            "images": [
                "8",
                0
            ]
        }
    }
}
"""



In [ ]:
prompt = json.loads(prompt_text)
prompt["6"]["inputs"]["text"] = "masterpiece best quality man"
prompt["3"]["inputs"]["seed"] = 5

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))
images = get_images(ws, prompt)

In [ ]:
from PIL import Image
import io
view_images=[]
for node_id in images:
    for image_data in images[node_id]:
        image = Image.open(io.BytesIO(image_data))
        view_images.append(image)
        
for view_image in view_images:
    image.show()   

### 生视频test

In [5]:
import time
import json
WORKING_DIR="/home/ec2-user/SageMaker/comfyui_output"
def show_gif(fname):
    import base64
    from IPython import display
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return display.HTML(f'<img src="data:image/gif;base64,{b64}" />')


prompt_json_file="./workflow_api_magic_animate.json"
with open(prompt_json_file) as f:
  prompt = json.load(f)

print(type(prompt))
ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))
start_dt=time.time()
images = get_images(ws, prompt)
end_dt=time.time()
print("time elapse:{:.6f} seconds".format(end_dt-start_dt))

for node_id in images:
    for image_data in images[node_id]:
        from PIL import Image
        import io
        GIF_LOCATION = "{}/Comfyui_{}.gif".format(WORKING_DIR, node_id)
        print(GIF_LOCATION)
        with open(GIF_LOCATION, "wb") as binary_file:
            # Write bytes to file
            binary_file.write(image_data)

        #show_gif(GIF_LOCATION)

<class 'dict'>
{'1': {'inputs': {'ckpt_name': 'v1-5-pruned-emaonly.safetensors'}, 'class_type': 'CheckpointLoaderSimple'}, '3': {'inputs': {'model_name': 'mm_sd_v15_v2.ckpt', 'beta_schedule': 'sqrt_linear (AnimateDiff)', 'motion_scale': 1, 'apply_v2_models_properly': True, 'model': ['1', 0], 'context_options': ['4', 0]}, 'class_type': 'ADE_AnimateDiffLoaderWithContext'}, '4': {'inputs': {'context_length': 16, 'context_stride': 1, 'context_overlap': 4, 'context_schedule': 'uniform', 'closed_loop': False}, 'class_type': 'ADE_AnimateDiffUniformContextOptions'}, '5': {'inputs': {'vae_name': 'vae-ft-mse-840000-ema-pruned.safetensors'}, 'class_type': 'VAELoader'}, '6': {'inputs': {'text': '1girl, solo, outdoor, autumn season, golden leaves, red leaves, falling leaves, black eyes, upper body, from side', 'clip': ['1', 1]}, 'class_type': 'CLIPTextEncode'}, '7': {'inputs': {'text': '(worst quality, low quality:1.4)', 'clip': ['1', 1]}, 'class_type': 'CLIPTextEncode'}, '8': {'inputs': {'sampling